In [36]:
import pandas as pd
import json
import os
from datetime import datetime, timedelta
import glob
import warnings
warnings.filterwarnings('ignore')

In [37]:
#date

date = ['20231002', '20231003', '20231004', '20231005', '20231006', '20231007', 
        '20231008', '20231009', '20231010', '20231011', '20231012', '20231013', 
        '20231014', '20231015', '20231016', '20231017', '20231018', '20231019', 
        '20231020', '20231025', 
        '20231026', '20231027', '20231028', '20231029', '20231030', '20231031', 
        '20231101', '20231102', '20231103', '20231104', '20231105', '20231106', 
        '20231107', '20231108', '20231109', '20231110', '20231111', '20231112', 
        '20231113', '20231114', '20231115', '20231116', '20231117', '20231118', 
        '20231119', '20231120', '20231121', '20231122', '20231123', '20231124', 
        '20231125', '20231126']

In [38]:
# 假设您的文本文件路径为 'path_to_your_txt_file.txt'
file_path_x = '/Users/liaomingyou/git/html.2023.final.data/sno_test_set.txt'

# 初始化一个空列表来存储读取的数据
station_list = []

# 打开并读取文件
with open(file_path_x, 'r') as file:
    for line in file:
        # 将每一行的内容添加到列表中
        # 使用 line.strip() 来去除每行末尾的换行符 '\n'
        station_list.append(line.strip())

# 现在 data_list 包含了文本文件中的每一行数据
print(station_list)

['500101001', '500101002', '500101003', '500101004', '500101005', '500101006', '500101007', '500101008', '500101009', '500101010', '500101013', '500101014', '500101015', '500101018', '500101019', '500101020', '500101021', '500101022', '500101023', '500101024', '500101025', '500101026', '500101027', '500101028', '500101029', '500101030', '500101031', '500101032', '500101033', '500101034', '500101035', '500101036', '500101037', '500101038', '500101039', '500101040', '500101041', '500101042', '500101091', '500101092', '500101093', '500101094', '500101114', '500101115', '500101123', '500101166', '500101175', '500101176', '500101181', '500101184', '500101185', '500101188', '500101189', '500101190', '500101191', '500101193', '500101199', '500101209', '500101216', '500101219', '500105066', '500106002', '500106003', '500106004', '500119043', '500119044', '500119045', '500119046', '500119047', '500119048', '500119049', '500119050', '500119051', '500119052', '500119053', '500119054', '500119055'

In [39]:
total_data = pd.DataFrame()

for i in date:
    for j in station_list:
        try:
            file_path = f'/Users/liaomingyou/git/html.2023.final.data/release/{i}/{j}.json'
            with open(file_path, 'r') as file:
                data = json.load(file)
                        
            df = pd.DataFrame(list(data.values()), index=data.keys())
            df_reset = df.reset_index()
            df_reset.rename(columns={'index': 'time'}, inplace=True)
            df_sbi = df_reset[['time','sbi']]
            df_sbi['sbi'] = df_sbi['sbi'].interpolate(method='linear', limit_direction='both', limit=4) # 最多上下四筆缺值補充，其他就清掉。
            df_sbi = df_sbi.dropna()
            
            station_name = os.path.basename(file_path).replace('.json', '')
            df_sbi['station_name'] = station_name
            df_sbi['date'] = i
            
            df_sbi['select_time'] = pd.to_datetime(df_sbi['time'], format='%H:%M')

            # 筛选出每20分钟的数据点
            df_selected = df_sbi[df_sbi['select_time'].dt.minute % 20 == 0]
            df_selected = df_selected.drop(['select_time'],axis=1)
            
            total_data = pd.concat([total_data, df_selected], ignore_index=True)
                
        except:
            pass

In [40]:
total_data

,time,sbi,station_name,date
0,00:00,12.0,500101001,20231002
1,00:20,11.0,500101001,20231002
2,00:40,12.0,500101001,20231002
3,01:00,8.0,500101001,20231002
4,01:20,8.0,500101001,20231002
...,...,...,...,...
385723,22:20,6.0,500119091,20231126
385724,22:40,4.0,500119091,20231126
385725,23:00,4.0,500119091,20231126
385726,23:20,4.0,500119091,20231126
